In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import urllib
import re
from IPython.display import display, HTML
import HTMLParser
import itertools
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2

# Start an SQL database

In [ ]:
# Create a Postgresql
dbname = 'art_kickstarter2'
username = raw_input() ##change user name to match with PostgreSQL
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print engine.url 
if not database_exists(engine.url):
    create_database(engine.url)
print 'Database exist? '+ str(database_exists(engine.url))


In [27]:
con = None
con = psycopg2.connect(database = dbname, user = username)

cursor = con.cursor()
command = "DROP TABLE IF EXISTS art_table;"
cursor.execute(command)
con.commit()
cursor.close()


# Scraper protytpe(s) (to be updated)

In [ ]:
def scraper_prototype1(proj_info,proj_prefix):
    proj_info_split = proj_info.split('"')
    for items in proj_info_split:
        if len(items)<2:
            proj_info_split.remove(items)

    proj_info_split = map(lambda x: x.lstrip(':'), proj_info_split) 
    proj_info_split = map(lambda x: x.rstrip(','), proj_info_split) 
    proj_info_dict = dict(itertools.izip_longest(*[iter(proj_info_split)] * 2, fillvalue=""))
    proj_info_dict = {proj_prefix+'_'+k: v for k,v in proj_info_dict.items()}
    #proj_info_dict = {'project_'+k: v for k,v in proj_info_dict.items()}
    
    return proj_info_dict


# Loading the project info on a page using a straper prototype and save into database.

In [ ]:
for pageno in range(10):
    r = urllib.urlopen('https://www.kickstarter.com/discover/advanced?state=live&category_id=1&sort=popularity&seed=2506137&page='+str(pageno)).read()
    soup = BeautifulSoup(r, 'html.parser')

    kk = soup.find_all('div', class_="js-react-proj-card")
    len(kk)

    for i in range(len(kk)):
        whole_string = str(kk[i]).replace('&amp','').replace('&quot;','')

        # === Overall project info ===
        try:
            project_info = '"name"'+ re.findall('"name"(.+?)"creator"',whole_string)[0]
            project_info_dict = scraper_prototype1(project_info,'project')
            df1 = pd.DataFrame.from_dict(project_info_dict,orient='index').transpose()

            # === Scrap creater string ===
            creator_string = re.findall(r'\"creator\"\:\{(.+?)\}',whole_string)
            creator_info_dict = scraper_prototype1(creator_string[0],'creator')
            df2 = pd.DataFrame.from_dict(creator_info_dict,orient='index').transpose()

            # === Scrap location string ===
            location_string = re.findall(r'\"location\"\:\{(.+?)\}',whole_string)
            location_info_dict = scraper_prototype1(location_string[0],'location')
            df3 = pd.DataFrame.from_dict(location_info_dict,orient='index').transpose()

            # === Scrap profile string ===
            profile_string = re.findall(r'\"profile\"\:\{(.+?)\}',whole_string)
            profile_info_dict = scraper_prototype1(profile_string[0],'profile')
            df4 = pd.DataFrame.from_dict(profile_info_dict,orient='index').transpose()

            df = pd.concat((df1,df2,df3,df4),axis=1)
            #display(df)

            df.to_sql('art_table', engine, if_exists='append')
        except:
            print 'Exception that scraper 1 failed. pageno = '+str(pageno)+'; i='+str(i)




# Check what's on the SQL database

In [30]:
sql_query = """
SELECT * FROM art_table;
"""
birth_data_from_sql = pd.read_sql_query(sql_query,con)

birth_data_from_sql.shape

(96, 61)

In [31]:
birth_data_from_sql.head(20)

,index,project_pledged,project_name,project_currency_symbol,project_currency,project_state_changed_at,project_disable_communication,project_country,project_state,project_blurb,...,profile_project_id,profile_text_color,profile_link_text_color,profile_link_url,profile_feature_image_attributes,profile_background_color,profile_state_changed_at,profile_image_urls,profile_background_image_opacity,profile_default
0,0,29628.0,Ai Weiwei: Good Fences Make Good Neighbors,$,USD,1503316228,false,US,live,Public Art Fund ; Ai Weiwei collaborate on a p...,...,3085789,null,null,null,{,null,1500998996,{,0.8,https://ksr-ugc.imgix.net/assets/017/805/547/c...
1,0,23397.0,Vibe Bottle - Kickstarter ver 2.0,$,USD,1503421895,false,US,live,"Vibe Bottle, art you can drink from! Refillabl...",...,3110397,null,null,null,{,null,1502897118,{,0.8,https://ksr-ugc.imgix.net/assets/017/944/010/b...
2,0,12274.0,The Sketching Tin,£,GBP,1502304049,false,GB,live,A portable kit for drawing ; painting. Small e...,...,3091978,null,null,null,{,null,1501498018,{,0.8,https://ksr-ugc.imgix.net/assets/017/807/374/3...
3,0,13388.0,Dinosaurs of the Wild West,$,USD,1503298926,false,US,live,A collection of illustrations depicting the ev...,...,3072593,null,null,null,{,null,1499977577,{,0.8,https://ksr-ugc.imgix.net/assets/017/497/408/9...
4,0,39227.0,TMNT ACTION FIGURE ENCYCLOPEDIA Hardcover and ...,$,USD,1502806674,false,US,live,An unofficial guide to nearly every TMNT actio...,...,3107049,null,null,null,{,null,1502650227,{,0.8,https://ksr-ugc.imgix.net/assets/017/904/674/7...
5,0,10071.0,Intergalac-treats: Galaxy-themed Enamel Pins a...,$,USD,1501096440,false,US,live,Welcome to Intergalac-treats! As the name sugg...,...,3078247,null,null,null,{,null,1500421192,{,0.8,https://ksr-ugc.imgix.net/assets/017/563/248/7...
6,0,29628.0,Ai Weiwei: Good Fences Make Good Neighbors,$,USD,1503316228,false,US,live,Public Art Fund ; Ai Weiwei collaborate on a p...,...,3085789,null,null,null,{,null,1500998996,{,0.8,https://ksr-ugc.imgix.net/assets/017/805/547/c...
7,0,23397.0,Vibe Bottle - Kickstarter ver 2.0,$,USD,1503421895,false,US,live,"Vibe Bottle, art you can drink from! Refillabl...",...,3110397,null,null,null,{,null,1502897118,{,0.8,https://ksr-ugc.imgix.net/assets/017/944/010/b...
8,0,12274.0,The Sketching Tin,£,GBP,1502304049,false,GB,live,A portable kit for drawing ; painting. Small e...,...,3091978,null,null,null,{,null,1501498018,{,0.8,https://ksr-ugc.imgix.net/assets/017/807/374/3...
9,0,13388.0,Dinosaurs of the Wild West,$,USD,1503298926,false,US,live,A collection of illustrations depicting the ev...,...,3072593,null,null,null,{,null,1499977577,{,0.8,https://ksr-ugc.imgix.net/assets/017/497/408/9...
